In [15]:
import pandas as pd
import numpy as np
import time
sample_path = '../data/5.sampled/'
k_mean_path = '../data/6.K-mean/'
mab_path = '../data/7.MAB/'

In [1]:
from bandits.core.contextual_bandit import run_contextual_bandit
from bandits.algorithms.linear_full_posterior_sampling import LinearFullPosteriorSampling
from bandits.data.data_sampler import  remove_underrepresented_classes

In [2]:
#결과출력함
def display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, name):
    """Displays summary statistics of the performance of each algorithm."""

    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print('{} bandit completed after {} seconds.'.format(
        name, time.time() - t_init))
    print('---------------------------------------------------')

    performance_pairs = []
    for j, a in enumerate(algos):
        performance_pairs.append((a.name, np.sum(h_rewards[:, j])))
    performance_pairs = sorted(performance_pairs,
                             key=lambda elt: elt[1],
                             reverse=True)
    for i, (name, reward) in enumerate(performance_pairs):
        print('{:3}) {:20}| \t \t total reward = {:10}.'.format(i, name, reward))

    print('---------------------------------------------------')
    print('Optimal total reward = {}.'.format(np.sum(opt_rewards)))
    print('Frequency of optimal actions (action, frequency):')
    print([[elt, list(opt_actions).count(elt)] for elt in set(opt_actions)])
    print('---------------------------------------------------')
    print('---------------------------------------------------')

In [3]:
 def clf_to_bandit_problem(contexts, labels, num_actions=None):
  """Normalize contexts and encode deterministic rewards."""

  if num_actions is None:
    num_actions = np.max(labels) + 1
  num_contexts = contexts.shape[0]

  # Due to random subsampling in small problems, some features may be constant
  #sstd = safe_std(np.std(contexts, axis=0, keepdims=True)[0, :])

  # Normalize features
  #contexts = ((contexts - np.mean(contexts, axis=0, keepdims=True)) / sstd)
  # One hot encode labels as rewards
  rewards = np.zeros((num_contexts, num_actions))
  rewards[np.arange(num_contexts), labels] = 1.0

  return contexts, rewards, (np.ones(num_contexts), labels)

In [22]:
sample_clicks = pd.read_csv(sample_path+'sampled_clicks.csv')
sample_orders = pd.read_csv(sample_path+'sampled_orders.csv')

In [23]:
merged_log = pd.merge(sample_clicks, sample_orders, how='left', on=['account_id', 'product_id'])

In [26]:
merged_log['reward'] = -1
merged_log.loc[merged_log[merged_log['order_id'].isna()].index ,'reward'] = 0
merged_log.loc[merged_log[merged_log['order_id'].notna()].index , 'reward'] = 1

In [27]:
interaction = merged_log[['account_id', 'product_id', 'reward', 'request_date_time']]
interaction.to_csv(mab_path+'interaction.csv', index=None)

In [28]:
sample_products = pd.read_csv(k_mean_path+'kmeans_product_onehotcol.csv')
sample_users = pd.read_csv(k_mean_path+'kmeans_user_onehotcol.csv')

In [29]:
user_inter = pd.merge(interaction, sample_users, how='left', on='account_id')
user_pro_inter = pd.merge(user_inter, sample_products, how='left', on='product_id')

In [30]:
user_pro_inter.drop(columns=['last_login_dts', 'product_name', 'published_at', 'maker_name'], inplace=True)
user_pro_inter.rename({'cluster_x': 'user_cluster', 'cluster_y' : 'book_cluster'}, axis=1, inplace=True)

In [31]:
user_pro_inter.rename(lambda x:'most_pref' + '_' + x.split('_')[-2] if x.endswith('x') else x,  axis=1, inplace=True)
user_pro_inter.rename(lambda x:x[:-2] if x.endswith('y') else x,  axis=1, inplace=True)

In [32]:
user_pro_inter.insert(2, 'user__cluster', user_pro_inter['user_cluster'])
user_pro_inter.insert(3, 'book__cluster', user_pro_inter['book_cluster'])
user_pro_inter.drop(['user_cluster','book_cluster'], axis=1, inplace=True)

In [34]:
user_pro_inter.to_csv(mab_path + 'sampled_mabdata.csv', index=None)

In [1]:
df_fin = pd.read_csv(mab_path + 'sampled_mabdata.csv')
df_fin.dropna(inplace=True)


NameError: name 'pd' is not defined

In [37]:
df_fin.sort_values('request_date_time', inplace=True)

In [38]:
df_fin

,account_id,product_id,user__cluster,book__cluster,reward,request_date_time,gender_M,gender_F,address_is_0,address_is_1,...,category_id_30,category_id_31,category_id_33,category_id_34,shop_price_0,shop_price_1,shop_price_2,shop_price_3,best200,bestcat24
25223,10046119,83491692,4.0,1.0,0,2019-12-01 00:26:09,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
25222,10046119,8013367,4.0,1.0,0,2019-12-01 00:31:14,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
31940,7085201,84498048,0.0,2.0,0,2019-12-01 01:23:01,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
31941,7085201,77709544,0.0,2.0,0,2019-12-01 01:24:58,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
32006,7085201,77709544,0.0,2.0,0,2019-12-01 01:25:19,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27879,10894947,42799081,0.0,2.0,1,2020-05-31 23:27:36,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
181,7560681,28313407,4.0,0.0,0,2020-05-31 23:46:33,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
280,7560681,67094938,4.0,4.0,0,2020-05-31 23:46:50,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
222,7560681,28313407,4.0,0.0,0,2020-05-31 23:47:18,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [40]:
(context, reward_1, )

Unique classes and their ratio of total: {0: 0.23851777295121046, 1: 0.3063516448588229, 2: 0.22135402871218943, 3: 0.004873655771573873, 4: 0.16623404142607406, 5: 0.06266885628012926}


In [42]:
opt_actions

array([1, 1, 2, ..., 4, 0, 4], dtype=int8)

In [8]:
dataset = np.hstack((contexts, rewards))
context_dim = contexts.shape[1]

In [9]:
class Hparams(object):
    def __init__(self, dicts):
        for k, v in dicts.items():
            self.__setattr__(k, v)

In [10]:
param_dicts = {'num_actions':num_actions,
 'context_dim': context_dim,
 'a0':6,
 'b0':6,
 'lambda_prior':0.25,
  'initial_pulls':2}

In [11]:
hparam = Hparams(param_dicts)

In [13]:
algos = [LinearFullPosteriorSampling('LinFullPost', hparam)]

In [16]:
t_init = time.time()
results = run_contextual_bandit(context_dim, num_actions, dataset, algos)

In [17]:
h_actions, h_rewards = results

In [19]:
display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, 'context')

---------------------------------------------------
---------------------------------------------------
context bandit completed after 2997.457048892975 seconds.
---------------------------------------------------
  0) LinFullPost         | 	 	 total reward =    36931.0.
---------------------------------------------------
Optimal total reward = 37754.0.
Frequency of optimal actions (action, frequency):
[[0, 9005], [1, 11566], [2, 8357], [3, 184], [4, 6276], [5, 2366]]
---------------------------------------------------
---------------------------------------------------
